
# 使用 LLM + Json Schema 进行文献信息抽取

## 1 背景与原理

在科研过程中，我们经常需要从众多文献中抽取关键信息，例如催化剂的合成方法、活性组分、表征结果等。传统做法是依靠人工逐篇阅读、整理，不仅效率低，而且容易遗漏。

近年来，大型语言模型（LLM, Large Language Model）的出现，为自动化或半自动化的信息抽取提供了可能。我们可以借助 LLM 对文献做定向提问，让模型从给定的文献摘要或文本片段中提取并以结构化的形式输出关键信息。

为了让 LLM 的输出更加可控、可验证，我们可以在提示词（prompt）中设定一个 **Json Schema**。这个 Schema 指定了输出结果应当包含哪些字段、字段类型、是否必填等信息。LLM 会根据我们在 prompt 中的指定，尝试输出符合该 Json Schema 格式的内容。

> **小结**  
> - **LLM** 可以理解我们的问题并生成文本，也可以根据我们的“规定”输出结构化数据。  
> - **Json Schema** 用于约束输出数据的结构，使得后续数据处理和分析更加方便。

---

## 2 乙炔加氢催化剂文献信息抽取示例

以下是一个专门针对“乙炔加氢催化剂文献信息抽取”的示例，包括了 **System Prompt** （给 LLM 的系统提示）和 **Json Schema** （用于规定输出结构）。当我们在调用 LLM 的 API 或者在对话界面中输入提示时，我们可以将这两个部分合并或分别输入（部分平台在 API 中支持 system 提示、user 提示）。

### 2.1 System Prompt 示例

我们先来看一下为“乙炔加氢催化剂信息抽取”制定的系统提示词。这个系统提示中详细说明了我们要如何抽取信息，以及需要输出 JSON 对象的格式（字段名称、字段含义等）：

```plaintext
Please use the following steps to extract information from the given literature on acetylene selective hydrogenation catalysts:
0. Collect basic information of this literature, including title, year, publication, list of authors (only corresponding authors) and their address.
1. Analyze whether the literature involves information on the synthesis, characterization, and evaluation of acetylene selective hydrogenation catalysts.
2. If it does, identify all relevant information on catalyst synthesis and evaluation.
3. Extract the following parameters:
   - Synthesis Method: Includes specific steps of catalyst synthesis (e.g., co-precipitation, sol-gel method, heat treatment, etc.) and heat treatment conditions (temperature, time, etc.).
   - Active Components and Promoters: Includes metal elements and their content.
   - Support Type: Type of support material and surface modification method.
   - Catalyst Structural Characteristics: Descriptions such as nanoparticles, single-atom sites, dual-atom sites, etc.
   - Heat Treatment Conditions: Temperature, time, atmosphere, etc.
   - Reaction Conditions: Acetylene concentration, temperature, pressure, etc.
   - Catalyst Activity, Selectivity, Stability: Activity (acetylene conversion), selectivity (ethylene selectivity), and stability (catalyst stability over time).
4. Organize the extracted information into a structured JSON format that complies with the given JSON Schema. Only report this json object.
```

### 2.2 Json Schema 示例

这是与上述 System Prompt 对应的 Json Schema 示例，定义了文献中要提取的信息字段和它们的类型、必填属性等。LLM 在输出时应该严格遵守这些字段规范。

In [ ]:
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "title": {"type": "string", "description": "The title of the article"},
    "year": {"type": "integer", "description": "The year of publication"},
    "journal_name": {"type": "string", "description": "The name of the journal"},
    "corresponding_authors": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": {"type": "string", "description": "The name of the corresponding author"},
          "address": {"type": "string", "description": "The address of the corresponding author"}
        },
        "required": ["name", "address"]
      },
      "description": "List of corresponding authors and their addresses"
    },
    "catalyst_name": {"type": "string", "description": "The name or identifier of the catalyst"},
    "synthesis_method": {"type": "string", "description": "Description of the synthesis method"},
    "active_components": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "element": {"type": "string", "description": "Element name"},
          "loading": {"type": "string", "description": "Loading amount"}
        },
        "required": ["element", "loading"]
      },
      "description": "Active components of the catalyst and their content"
    },
    "promoters": {"type": "array", "items": {"type": "string"}, "description": "Promoters"},
    "support": {"type": "string", "description": "Support material"},
    "heat_treatment": {
      "type": "object",
      "properties": {
        "temperature": {"type": "string", "description": "Temperature"},
        "duration": {"type": "string", "description": "Duration"},
        "atmosphere": {"type": "string", "description": "Atmosphere"}
      },
      "required": ["temperature", "duration", "atmosphere"]
    },
    "reaction_conditions": {
      "type": "object",
      "properties": {
        "acetylene_concentration": {"type": "string", "description": "Concentration"},
        "temperature": {"type": "string", "description": "Temperature"},
        "pressure": {"type": "string", "description": "Pressure"}
      },
      "required": ["acetylene_concentration", "temperature", "pressure"]
    },
    "catalyst_performance": {
      "type": "object",
      "properties": {
        "activity": {"type": "string", "description": "Activity"},
        "selectivity": {"type": "string", "description": "Selectivity"},
        "stability": {"type": "string", "description": "Stability"}
      },
      "required": ["activity", "selectivity", "stability"]
    }
  },
  "required": [
    "title", "year", "journal_name", "corresponding_authors", 
    "catalyst_name", "synthesis_method", "active_components", 
    "promoters", "support", "heat_treatment", 
    "reaction_conditions", "catalyst_performance"
  ]
}


## 3 如何让大语言模型帮助我们生成一个领域专属的 Json Schema

上面的示例适用于“乙炔加氢”领域。如果课题组的同学想将同样的思路应用到其他领域（如“甲酸分解催化剂研究”），可以让大语言模型协助生成初步的 Json Schema，再进行人工修改和完善。例如，我们可以向模型提出以下问题（Prompt）：

```plaintext
我研究甲酸分解催化剂，想要提取以下信息：文献的标题、作者、甲酸分解反应条件（包括温度、pH 值、溶剂类型）、催化剂合成方法、活性评价（比如产氢速率）、以及表征手段。请帮忙根据这些需求给出一个合适的 Json Schema，限定字段类型和必填属性。
```

这样，LLM 就可以给出一个针对“甲酸分解”领域的 Json Schema 示例。下面是一个可能的输出示例（仅供参考）：

In [ ]:
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "Formic Acid Decomposition Catalyst Metadata",
  "type": "object",
  "properties": {
    "title": {
      "type": "string",
      "description": "The title of the article"
    },
    "authors": {
      "type": "array",
      "description": "List of authors",
      "items": {
        "type": "string"
      }
    },
    "reaction_conditions": {
      "type": "object",
      "properties": {
        "temperature": {
          "type": "string",
          "description": "Reaction temperature, e.g., 80 °C"
        },
        "pH": {
          "type": "string",
          "description": "pH value during the reaction, e.g., pH=7"
        },
        "solvent": {
          "type": "string",
          "description": "Solvent used, e.g., water, methanol"
        }
      },
      "required": ["temperature", "pH", "solvent"]
    },
    "catalyst_synthesis_method": {
      "type": "string",
      "description": "Description of the synthesis method"
    },
    "performance_evaluation": {
      "type": "object",
      "properties": {
        "hydrogen_production_rate": {
          "type": "string",
          "description": "Rate of hydrogen production, e.g., 100 mL/min"
        }
      },
      "required": ["hydrogen_production_rate"]
    },
    "characterization": {
      "type": "array",
      "description": "List of characterization techniques",
      "items": {
        "type": "string",
        "description": "Technique name, e.g., XRD, TEM, IR, etc."
      }
    }
  },
  "required": [
    "title",
    "authors",
    "reaction_conditions",
    "catalyst_synthesis_method",
    "performance_evaluation",
    "characterization"
  ]
}

同样，你可以在 System Prompt 或 User Prompt 中加入对 Json Schema 的详细要求，让模型针对自己研究领域的关注点（例如“载体类型”、“金属前驱体”、“比表面积”等）进行输出。然后在后续对文献进行信息抽取时，将生成的 Json Schema 当作“标准模版”使用。

## 4 在更复杂场景下的展望

在更复杂的场景中，我们不仅仅想收集文献中的简单字段，还想表达更深层次的关系，例如“催化剂 A 的活性组分 B 在载体 C 上以 D 的方式分散，表现出的活性、选择性、稳定性如何。” 这时我们就可以在 Json Schema 中嵌入更具层级结构的定义，或者使用更加专业的 **Ontology**（本体）来约束和组织专业概念，诸如：

- 催化剂的层次结构：金属前驱体 -> 金属晶格 -> 金属/氧化物相 -> 载体的表面性质等
- 反应路径、产物分布等信息的嵌套
- 表征方法与对应的结果（如 XRD 显示什么晶相、TEM 观察到多少纳米尺寸等）

理论上，只要 LLM 理解并“记住”了我们在 Prompt 中给予的“Ontology”或“深层结构定义”，就可以输出带有层级化或嵌套关系的 JSON。示例（简化）如下：

In [ ]:
{
    "catalyst": {
      "components": [
        {
          "element": "Pd",
          "phase": "Pd0",
          "size": "3nm"
        },
        {
          "element": "CeO2",
          "phase": "Ce4+",
          "size": "10nm"
        }
      ],
      "structure": "Pd nanoparticles on CeO2 nanospheres"
    },
    "reaction_pathway": [
      {
        "reactant": "CH3OH",
        "intermediate": "HCHO",
        "product": "H2 and CO2"
      }
    ]
  }

当然，这就远比简单的字段提取要复杂得多，需要在 Prompt 中对字段命名、层级、描述方式等进行详细约束，且最好在后续有配套的自动化校验或人工审阅流程，保证数据的真实性与一致性。

总结
1. LLM + Json Schema 能够帮助我们快速从文献中提取结构化的信息，实现自动化或半自动化的数据采集与知识库构建。
2.	要想使用该方法，需要编写一个合适的 System Prompt（告诉 LLM 如何抽取信息）并提供一个 Json Schema（告诉 LLM 如何组织这些信息）。
3.	同学们可以使用已有示例或让 LLM 帮助自己“自动生成”适合自己研究领域的 Json Schema，再在此基础上进行人工修改、完善。
4.	如果对信息的层次结构和概念体系要求更高，可以进一步引入 Ontology 等高级概念，以便让 LLM 输出更加复杂、嵌套更深的结构化数据。

希望这份说明文档能为课题组同学带来灵感，帮助大家利用 LLM 提升科研效率。如果在使用过程中遇到问题，可以随时讨论和改进 Prompt 或 Schema，找到最契合课题需求的方案。祝大家科研顺利!